In [60]:
import pandas as pd
import numpy as np
import io
print(pd.ExcelFile(r"data\raw\TEMP_KRITUL_1991_2020_kasdieniai.xls").sheet_names)

df_temp = pd.read_excel(r"data\raw\TEMP_KRITUL_1991_2020_kasdieniai.xls", sheet_name="Temp_vidutinė")
df_prec = pd.read_excel(r"data\raw\TEMP_KRITUL_1991_2020_kasdieniai.xls", sheet_name="Krituliai")


['Temp_vidutinė', 'Temp_MIN', 'Temp_MAX', 'Krituliai']


In [79]:
pd.options.display.max_rows = 20

In [81]:
from decimal import Decimal, getcontext
getcontext().prec = 100

In [ ]:
temperatureDf = df_temp.drop(columns=df_temp.columns[-2:])
column_mapping = {
    "TVID" : "year", 
    "mėnuo" : "month", 
    "diena" : "day",
    'Biržai': 'birzu-ams',
    'Dotnuva': 'dotnuvos-ams',
    'Dūkštas': 'duksto-ams',
    'Kaunas': 'kauno-ams',
    'Kybartai': 'kybartu-ams',
    'Klaipėda': 'klaipedos-ams',
    'Laukuva': 'laukuvos-ams',
    'Lazdijai': 'lazdiju-ams',
    'Nida': 'nidos-ams',
    'Panevėžys': 'panevezio-ams',
    'Raseiniai': 'raseiniu-ams',
    'Šiauliai': 'siauliu-ams',
    'Šilutė': 'silutes-ams',
    'Telšiai': 'telsiu-ams',
    'Ukmergė': 'ukmerges-ams',
    'Utena': 'utenos-ams',
    'Varėna': 'varenos-ams',
    'Vilnius TV': 'vilniaus-ams',
    "Krit" : "year",
}


temperatureDf = temperatureDf.rename(columns=column_mapping)
precipitationDf = df_prec.drop(columns=df_prec.columns[-1:])
precipitationDf = precipitationDf.rename(columns=column_mapping)


def group_values(df, aggType):
    agg_functions = {c:aggType for c in df.columns[3:]}
    grouped_df = df.groupby(
        [
            "year",
            "month",
        ]
    )
    return grouped_df.aggregate(agg_functions)

def processDf(df):
    df = df.drop(df[df['year'] < 2004].index)
    return df
temperatureDf = processDf(temperatureDf)
precipitationDf = processDf(precipitationDf)

temperatureDf = group_values(temperatureDf,"mean").reset_index()
precipitationDf = group_values(precipitationDf, "sum").reset_index()


In [122]:
finalDf = pd.read_csv(r"data\attempt.csv")
municipality_map = pd.read_csv(r"data\municipalities_meteo_stations.csv")
l = []
for dfColNameCombo in [[temperatureDf, "oro_temp"], [precipitationDf, "kritutliu_kiekis"]]:
    for idx, row in finalDf.iterrows():
        station = municipality_map.loc[municipality_map["savivaldybe"] == row["name"], "stoties_kodas"]
        
        # to find unmapped municipalities

        if not len(station):
            if row["name"] not in l:
                l.append(row["name"])
                print("EROOR NX" + row["name"])
        else:
            pass
        
        #print(meteoDf["stoties_kodas"] == str(station))
        station = station.squeeze()
        meteoNum = dfColNameCombo[0].loc[(dfColNameCombo[0]["year"] == row["year"]) & (dfColNameCombo[0]["month"] == row["month"]), station]
        if not len(meteoNum):
            continue
        finalDf.at[idx, dfColNameCombo[1]] = meteoNum.squeeze()

In [117]:
finalDf.sample(10)

,name,year,month,infection,forest cover,population,area,oro_temp,juntamoji_temp,debesuotumas,slegis_juros_lyg,santyk_oro_dregme,kritutliu_kiekis,krituliu_kiekis
3901,Alytaus apskritis,2009,7,5,49.1,167603.0,5418.0,18.280645,NaN,NaN,NaN,NaN,NaN,130.2
5589,Ukmergės r. sav.,2011,9,8,32.4,40270.0,1395.0,13.460000,NaN,NaN,NaN,NaN,NaN,72.1
8178,Vilkaviškio r. sav.,2015,2,1,11.0,39969.0,1262.0,0.557143,-2.544940,68.409366,1016.312351,83.290179,10.3,10.3
7971,Kauno r. sav.,2014,11,6,32.5,85893.0,1496.0,2.760000,0.526667,80.024425,1023.020694,87.613889,29.5,29.5
260,Šilutės r. sav.,2004,7,5,22.1,53143.0,1682.0,15.958065,NaN,NaN,NaN,NaN,NaN,61.3
9740,Molėtų r. sav.,2018,2,1,31.6,18158.0,1367.0,-6.796429,-8.749851,79.203008,1021.648065,82.159226,19.0,19.0
625,Panevėžio apskritis,2005,1,8,27.3,284918.0,7878.0,-0.809677,NaN,NaN,NaN,NaN,NaN,46.1
6492,Lazdyjų r.,2012,12,0,NaN,NaN,NaN,-4.500000,NaN,NaN,NaN,NaN,NaN,45.5
647,Utenos apskritis,2005,1,0,32.8,175226.0,7191.0,-0.832258,NaN,NaN,NaN,NaN,NaN,50.0
9715,Marijampolės apskritis,2018,2,1,22.0,143567.0,4465.0,-5.075000,-8.551786,73.116071,1021.269643,83.211310,12.2,12.1


In [123]:
finalDf.to_csv(r"data\final.csv", index=None)